In [0]:
from pyspark.sql import SparkSession

In [0]:
spark = SparkSession.builder.appName("thebigdatashow.me")


In [0]:
spark

In [0]:

from pyspark.sql.window import Window
from pyspark.sql.functions import *
from pyspark.sql.types import *
class Transform:
    def __init__(self):
        pass
    def transform(self,inputDFs):
        pass

class FirstTransformation(Transform):
    def transform(self,inputDFs):
        inputdf = inputDFs.get("transactiondf")

        windowSpec = Window.partitionBy(col("customer_id")).orderBy(col("transaction_date"))

        transformdf = inputdf.withColumn("nextProductName",lead(col("product_name")).over(windowSpec))

        

        filterdf = transformdf.filter((col("product_name") == "iPhone") &
                                      (col("nextProductName") == "AirPods"))
        filterdf.orderBy(col("customer_id"),col("transaction_date")).display()


        #customerdf codes
        customerdata = inputDFs.get("customerdf")
        customerdata.show()

        #joining two dfs

        joined_df = filterdf.join(customerdata,"customer_id")

        return joined_df\
               .select('customer_id','customer_name','location')
               
        


        


In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import *
from pyspark.sql.types import *

class OnlyAirpodsAndIphone:
    def transform(self,inputdfs):
        inputdf = inputdfs.get("transactiondf")

        groupeddf = inputdf.groupBy(col("customer_id")).agg(collect_set(col("product_name")).alias("list_of_products"))

        filterdf = groupeddf.filter((array_contains(col("list_of_products"),"iPhone")) &
                                    (array_contains(col("list_of_products"),"AirPods")) &
                                     (size(col("list_of_products"))==2))
        customer_data = inputdfs.get("customerdf")

        joineddf = customer_data.join(broadcast(filterdf),"customer_id")

        return joineddf.select("customer_id","list_of_products","location")
                


